In [1]:
import requests
import os
import datetime
import zipfile
from tqdm import tqdm
import pandas as pd
import feather

class DataProcessor:
    def __init__(self, base_url, zip_download_dir, csv_extract_dir, feather_stored_dir):
        self.base_url = base_url
        self.zip_download_dir = zip_download_dir
        self.csv_extract_dir = csv_extract_dir
        self.feather_stored_dir = feather_stored_dir

    def download_file(self, url, destination):
        if os.path.exists(destination):
            print(f"File {destination} already exists. Skipping download.")
            return
        response = requests.get(url)
        if response.status_code == 200:
            with open(destination, 'wb') as file:
                file.write(response.content)

    def download_and_extract_and_feather_data(self, start_date, end_date):
        os.makedirs(self.zip_download_dir, exist_ok=True)
        os.makedirs(self.csv_extract_dir, exist_ok=True)
        os.makedirs(self.feather_stored_dir, exist_ok=True)
        download_bar = tqdm(total=(end_date - start_date).days + 1, desc="Downloading, Extracting, and Featherizing")

        for current_date in (start_date + datetime.timedelta(n) for n in range((end_date - start_date).days + 1)):
            date_str = current_date.strftime('%Y-%m-%d')
            zip_url = f'{self.base_url}ETHUSDT-aggTrades-{date_str}.zip'
            zip_filename = os.path.join(self.zip_download_dir, f'ETHUSDT-aggTrades-{date_str}.zip')

            self.download_file(zip_url, zip_filename)

            with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
                for file_info in zip_ref.infolist():
                    extracted_file_path = os.path.join(self.csv_extract_dir, file_info.filename)

                    if not os.path.exists(extracted_file_path):
                        zip_ref.extract(file_info, path=self.csv_extract_dir)
                    else:
                        print(f"File {extracted_file_path} already exists. Skipping extraction.")

                    # Transfer to Feather format
                    csv_file_path = extracted_file_path
                    feather_file_path = os.path.join(self.feather_stored_dir, f"{os.path.splitext(file_info.filename)[0]}.feather")
                    self.transfer_to_feather(csv_file_path, feather_file_path)

            download_bar.update(1)
        download_bar.close()

    def transfer_to_feather(self, csv_file_path, feather_file_path):
        if not os.path.exists(feather_file_path):
            df = pd.read_csv(csv_file_path)
            df = self._convert_to_seconds(df)
            df.to_feather(feather_file_path)
        else:
            print(f"Feather file for {csv_file_path} already exists. Skipping transfer.")

    def _convert_to_seconds(self, df):
        df['utc_time'] = pd.to_datetime(df['transact_time'], unit='ms')
        df.set_index('utc_time', inplace=True)

        resampled_df = df.resample('S').agg({
            'agg_trade_id': 'first',
            'price': 'ohlc',
            'quantity': 'sum',
            'first_trade_id': 'first',
            'last_trade_id': 'last',
            'is_buyer_maker': 'last',
        })
        resampled_df.reset_index(inplace=True)
        resampled_df.columns = resampled_df.columns.droplevel()
        resampled_df.fillna(method='ffill', inplace=True)
        resampled_df.fillna(method='bfill', inplace=True)

        return resampled_df

    def combine_from_feather_to_df(self, data_dir, start_date, end_date):
        data_files = [file for file in os.listdir(data_dir) if file.endswith('.feather')]
        data_files.sort()
        combined_df = None
        progress_bar = tqdm(total=len(data_files), desc="Combining Feather Files")

        for data_file in data_files:
            date_str = '-'.join(data_file.split('-')[-3:]).replace('.feather', '')
            file_date = datetime.datetime.strptime(date_str, '%Y-%m-%d').date()

            if start_date <= file_date <= end_date:
                data_path = os.path.join(data_dir, data_file)
                df = pd.read_feather(data_path)
                # df = self._convert_to_seconds(df)
                combined_df = pd.concat([combined_df, df]) if combined_df is not None else df
            progress_bar.update(1)

        progress_bar.close()
        return combined_df

    def delete_all_data(self):
        os.system(f'rm -rf {self.zip_download_dir}')
        os.system(f'rm -rf {self.csv_extract_dir}')
        os.system(f'rm -rf {self.feather_stored_dir}')

# Example usage:
base_url = 'https://data.binance.vision/data/futures/um/daily/aggTrades/ETHUSDT/'
zip_download_dir = '/allah/freqtrade/Orange_project/aggTrades/binance_aggTrades'
csv_extract_dir = '/allah/freqtrade/Orange_project/aggTrades/decompressed_csv'
feather_stored_dir = '/allah/freqtrade/Orange_project/aggTrades/feather_data'

data_processor = DataProcessor(base_url, zip_download_dir, csv_extract_dir, feather_stored_dir)
start_date = datetime.date(2023, 1, 1)
end_date = datetime.date(2023, 10, 16)

# Download, extract, and featherize data
# data_processor.download_and_extract_and_feather_data(start_date, end_date)

# # Combine selected Feather files to a DataFrame
combined_df = data_processor.combine_from_feather_to_df(feather_stored_dir, datetime.date(2023, 10, 14), datetime.date(2023, 10, 15))

# # Perform operations with combined_df as needed.


Combining Feather Files: 100%|██████████| 361/361 [00:00<00:00, 2156.19it/s]


In [2]:
import pandas as pd
from datetime import datetime
import feather

# combined_df = data_processor.combine_from_feather_to_df(feather_stored_dir, datetime.date(2023, 4, 13), datetime.date(2023, 5, 16))

def format_and_save_dataframe(input_df, output_path):
    # Create a copy of the input DataFrame
    df_formatted = input_df.copy()

    # Define the end time for your time series
    end_time = datetime.strptime('2023-10-16 23:00:00', '%Y-%m-%d %H:%M:%S')

    # Create a new index based on a range with 1-minute frequency
    new_index = pd.date_range(end=end_time, periods=len(df_formatted), freq='1T')

    # Assign the new index to the DataFrame and rename columns
    df_formatted.index = new_index
    df_formatted = df_formatted.rename(columns={'': 'real_1s'})

    # Reset the index to make the 'date' column a regular column
    df_formatted = df_formatted.reset_index()

    # Rename and format columns to match the target format
    df_formatted['date'] = df_formatted['index']
    df_formatted['volume'] = df_formatted['quantity']
    # df_formatted['date'] = pd.to_datetime(df_formatted['date']).dt.strftime('%Y-%m-%d %H:%M:%S') + '+00:00'
    df_formatted['date'] = pd.to_datetime(df_formatted['date'])

    df_formatted_temp = df_formatted[['date','real_1s','open', 'high', 'low', 'close', 'volume', 'first_trade_id', 'last_trade_id', 'agg_trade_id', 'is_buyer_maker']]

    df_formatted = df_formatted[['date', 'open', 'high', 'low', 'close', 'volume']]

    # Save the formatted DataFrame to a Feather file
    # feather.write_dataframe(df_formatted, output_path)
    df_formatted.to_feather(
            output_path, compression_level=9, compression='lz4')
    return df_formatted_temp
# Usage
input_df = combined_df.copy()  # Replace with your actual DataFrame
output_path = '/allah/freqtrade/user_data/data/binance/futures/BTC_USDT_USDT-1m-futures.feather'
df_formatted_temp = format_and_save_dataframe(input_df, output_path)
df_formatted_temp[['real_1s']].to_feather('/allah/freqtrade/user_data/strategies/real_1s.feather')




In [3]:
# Make a copy of the DataFrame to work with
df_test = df_formatted_temp.copy()

# Drop the 'date' column since it's no longer needed
df_test.drop("date", axis=1, inplace=True)

# Convert the 'real_1s' column to datetime objects
df_test['real_1s'] = pd.to_datetime(df_test['real_1s'])

# Sort the DataFrame by 'real_1s' for chronological order
df_test = df_test.sort_values(by='real_1s')

# Define a function to calculate the cumulative sum for each minute
def calculate_aggregated_sum(df):
    return df['volume'].cumsum()

# Apply the function to the DataFrame and store the results
df_test['aggregate_volume_sum'] = df_test.groupby(df_test['real_1s'].dt.strftime('%Y-%m-%d %H:%M')).apply(calculate_aggregated_sum).values

# Define a function to calculate the sum of volumes for each minute
def calculate_volumes_sum(df):
    return df['volume'].sum()

# Group the DataFrame by minute and apply the function
df_1m_aggregated = df_test.groupby(df_test['real_1s'].dt.strftime('%Y-%m-%d %H:%M')).apply(calculate_volumes_sum)

# Create a new DataFrame from the result Series and compute the sum of volumes over the last 5 minutes
df_1m_aggregated = pd.DataFrame(df_1m_aggregated, columns=['current_minute_volume'])
df_1m_aggregated['last_5_minutes_volumes_sum'] = df_1m_aggregated['current_minute_volume'].rolling(6).sum() - df_1m_aggregated['current_minute_volume']
df_1m_aggregated.reset_index(inplace=True)
df_test['last_5_minutes_volumes_sum'] = df_test['real_1s'].dt.strftime('%Y-%m-%d %H:%M').map(df_1m_aggregated.set_index('real_1s')['last_5_minutes_volumes_sum'])



In [12]:
condition = df_test['aggregate_volume_sum'] > df_test['last_5_minutes_volumes_sum']
df_test[condition]

,real_1s,open,high,low,close,volume,first_trade_id,last_trade_id,agg_trade_id,is_buyer_maker,aggregate_volume_sum,last_5_minutes_volumes_sum,entry
4057,2023-10-14 01:07:41,1550.55,1550.56,1550.37,1550.37,247.317,3.277837e+09,3.277837e+09,1.394104e+09,1.0,1871.438,1777.262,0
4058,2023-10-14 01:07:42,1550.36,1550.50,1550.36,1550.50,180.901,3.277837e+09,3.277838e+09,1.394104e+09,0.0,2052.339,1777.262,0
4059,2023-10-14 01:07:43,1550.50,1550.50,1550.49,1550.49,2.918,3.277838e+09,3.277838e+09,1.394104e+09,1.0,2055.257,1777.262,0
4060,2023-10-14 01:07:44,1550.50,1550.50,1550.49,1550.50,5.794,3.277838e+09,3.277838e+09,1.394104e+09,0.0,2061.051,1777.262,0
4061,2023-10-14 01:07:45,1550.49,1550.50,1550.49,1550.50,0.415,3.277838e+09,3.277838e+09,1.394104e+09,0.0,2061.466,1777.262,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
170507,2023-10-15 23:21:55,1553.50,1553.51,1553.50,1553.50,0.034,3.279445e+09,3.279445e+09,1.394621e+09,1.0,3194.924,2990.052,0
170508,2023-10-15 23:21:56,1553.51,1553.51,1553.50,1553.50,0.037,3.279445e+09,3.279445e+09,1.394621e+09,1.0,3194.961,2990.052,0
170509,2023-10-15 23:21:57,1553.51,1553.51,1553.51,1553.51,0.010,3.279445e+09,3.279445e+09,1.394621e+09,0.0,3194.971,2990.052,0
170510,2023-10-15 23:21:58,1553.50,1553.50,1553.50,1553.50,0.022,3.279445e+09,3.279445e+09,1.394621e+09,1.0,3194.993,2990.052,0


In [13]:
# Initialize the 'entry' column with zeros
df_test['entry'] = 0

# Find the rows where the condition is met
condition = df_test['aggregate_volume_sum'] > df_test['last_5_minutes_volumes_sum']
condition = condition & (condition != condition.shift(1))

# Set 'entry' to 1 for the first occurrence within each minute
df_test.loc[condition, 'entry'] = 1


In [17]:
df_test[df_test['entry'] == 1]

,real_1s,open,high,low,close,volume,first_trade_id,last_trade_id,agg_trade_id,is_buyer_maker,aggregate_volume_sum,last_5_minutes_volumes_sum,entry
4057,2023-10-14 01:07:41,1550.55,1550.56,1550.37,1550.37,247.317,3.277837e+09,3.277837e+09,1.394104e+09,1.0,1871.438,1777.262,1
7006,2023-10-14 01:56:50,1553.23,1554.25,1553.23,1554.08,2831.420,3.277859e+09,3.277860e+09,1.394110e+09,1.0,3322.187,1916.904,1
7039,2023-10-14 01:57:23,1554.87,1554.88,1554.87,1554.88,145.461,3.277864e+09,3.277864e+09,1.394111e+09,0.0,6491.872,6440.294,1
9813,2023-10-14 02:43:37,1553.19,1553.67,1552.81,1553.42,1065.547,3.277897e+09,3.277897e+09,1.394120e+09,1.0,4827.465,4492.144,1
13052,2023-10-14 03:37:36,1553.19,1553.45,1553.19,1553.45,161.946,3.277925e+09,3.277926e+09,1.394128e+09,0.0,1717.081,1597.634,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
166380,2023-10-15 22:13:08,1561.35,1561.69,1561.24,1561.51,1705.282,3.279352e+09,3.279352e+09,1.394593e+09,0.0,12253.829,11114.043,1
168406,2023-10-15 22:46:54,1556.74,1556.80,1556.33,1556.59,651.862,3.279383e+09,3.279384e+09,1.394602e+09,0.0,4650.352,4362.250,1
168449,2023-10-15 22:47:37,1554.66,1554.67,1553.59,1553.65,1629.115,3.279391e+09,3.279393e+09,1.394604e+09,1.0,9959.023,9667.986,1
168939,2023-10-15 22:55:47,1553.68,1554.77,1553.68,1554.36,2312.567,3.279418e+09,3.279419e+09,1.394612e+09,1.0,9965.638,9515.463,1
